In [0]:
%run ./00_config

In [0]:
from pyspark.sql.functions import col, to_timestamp, current_timestamp

try:
    print("⏳ Iniciando procesamiento de la Capa Silver...")
    
    # 1. LECTURA DE CAPA BRONZE
    # Databricks leerá la variable BRONCE_TAXI_TABLE de tu archivo 00_config
    df_viajes = spark.table(BRONCE_TAXI_TABLE)
    
    # 2. LIMPIEZA Y NORMALIZACIÓN (Reglas de Calidad Senior)
    print("🧹 Limpiando y estandarizando datos (Casteos, Filtros y Duplicados)...")
    
    # Aplicamos dropDuplicates definiendo una llave lógica compuesta antes de castear
    df_silver = df_viajes.dropDuplicates(["hvfhs_license_num", "pickup_datetime","Dropoff_datetime", "PULocationID","DOLocationID","driver_pay","request_datetime"]) \
                         .withColumn("pickup_datetime", to_timestamp(col("pickup_datetime"))) \
                         .withColumn("dropoff_datetime", to_timestamp(col("dropoff_datetime"))) \
                         .withColumn("base_passenger_fare", col("base_passenger_fare").cast("decimal(10,2)")) \
                         .withColumn("tolls", col("tolls").cast("decimal(10,2)")) \
                         .withColumn("sales_tax", col("sales_tax").cast("decimal(10,2)")) \
                         .withColumn("congestion_surcharge", col("congestion_surcharge").cast("decimal(10,2)")) \
                         .withColumn("airport_fee", col("airport_fee").cast("decimal(10,2)")) \
                         .withColumn("tips", col("tips").cast("decimal(10,2)")) \
                         .withColumn("bcf", col("bcf").cast("decimal(10,2)")) \
                         .filter(col("PULocationID").isNotNull()) \
                         .filter(col("dropoff_datetime") > col("pickup_datetime")) # Filtramos viajes ilógicos

    # 3. ENRIQUECIMIENTO (JOIN con Zonas)
    print("🗺️ Enriqueciendo datos con el catálogo de zonas...")
    
    # Validación: Verificamos si la tabla de zonas existe antes de cruzar
    if spark.catalog.tableExists(BRONCE_ZONES_TABLE):
        df_zonas = spark.table(BRONCE_ZONES_TABLE)
        
        # Hacemos un LEFT JOIN para no perder viajes si la zona no aparece en el catálogo
        df_final_silver = df_silver.join(
            df_zonas, 
            df_silver.PULocationID == df_zonas.LocationID, 
            "left"
        ).select(
            df_silver["*"],
            col("Borough").alias("pickup_borough"),
            col("Zone").alias("pickup_zone")
        )
    else:
        print(f"⚠️ Tabla '{BRONCE_ZONES_TABLE}' no encontrada. Omitiendo JOIN de zonas por ahora.")
        df_final_silver = df_silver

    # 4. AUDITORÍA Y PERSISTENCIA
    # Agregamos la marca de tiempo exacta de cuándo pasó por la capa Silver
    df_final_silver = df_final_silver.withColumn("silver_processing_timestamp", current_timestamp())

    print(f"💾 Guardando datos refinados en la tabla: {SILVER_TAXI_TABLE}...")
    df_final_silver.write.mode("overwrite") \
                   .option("overwriteSchema", "true") \
                   .saveAsTable(SILVER_TAXI_TABLE)

    print("✅ Capa Silver completada con éxito.")
    
    # Mostramos el esquema resultante y los primeros 5 registros
    df_final_silver.printSchema()
    display(df_final_silver.limit(5))

except Exception as e:
    print(f"❌ Error en Capa Silver: {str(e)}")

In [0]:
# --- Validación de IDs eliminados en el proceso Silver ---
bronze_table = "workspace.default.bronze_nyc_taxi_2025_enero"
silver_table = "workspace.default.silver_nyc_taxi"

df_bronze = spark.table(bronze_table)
df_silver = spark.table(silver_table)

# Identificamos los IDs que estaban en Bronze pero no en Silver
df_ids_eliminados = df_bronze.join(
    df_silver.select("id"), 
    df_bronze.id == df_silver.id, 
    "left_anti"
).select(df_bronze.id.alias("id_eliminado"))

display(df_ids_eliminados)

In [0]:
%sql
-- Validacion de por que se eliminaron los id's, con el fin de garantizar el correcto proceso de limpieza de los datos --
-- Estos registros se eliminan por que en las columnas pickup_datetime y dropoff_datetime tienen la misma fecha y hora y en el analisis se calcula la distancia de recorrido de los servicios --

select * from bronze_nyc_taxi_2025_enero where id in ('95a57e69-9b81-4a1f-86c9-55838468c369',
'f532a55f-9a65-4ad1-b870-2e0d97782e73',
'fde4bd57-ba81-42a1-b6f1-39a875b08650',
'219f39de-6bdb-4a58-8c5e-0373afd17216'
)

In [0]:
%sql
select * from bronze_nyc_taxi_2025_enero where hvfhs_license_num = 'HV0003'
and pickup_datetime = '2025-01-03T18:45:58.000+00:00'
and PULocationID = 130
and DOLocationID = 134
and driver_pay = 6.72